# Wise Case study. Presented by Maxim Keller

## Library Imports and Constant Declarations

In [1]:
import pandas as pd

In [2]:
START_DATE = '2022-04-01'
END_DATE = '2023-08-01'

## Data input, data clearance
### Customers tab

In [3]:
# import DataFrame
customer = pd.read_excel('Dummy Data.xlsx', sheet_name="Customer", header=1)

In [4]:
#As the initial file wasn’t properly prepared, 
#it is important to prepare the DataFrame correctly.
#keep only needed columns and rename to exclude whitespaces

columns = ['Customer_Id ', 
         'Customer_Type',
         'Current_Address_Country',
         'Customer_Since_Date'
        ]
customer = customer[columns]
customer = customer.rename(columns={'Customer_Id ': 'Customer_Id'})

In [5]:
# Ensure that all the formats are correct
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Customer_Id              100 non-null    float64       
 1   Customer_Type            96 non-null     object        
 2   Current_Address_Country  100 non-null    object        
 3   Customer_Since_Date      100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 3.2+ KB


In [6]:
# According to the existing inputs, 
# we can change the format of the Customer_Id column.
customer['Customer_Id'] = customer['Customer_Id'].astype(int)

In [7]:
# As an initial step in data cleaning, 
# we will use the describe() function.
customer.describe(include='all',datetime_is_numeric=True)

,Customer_Id,Customer_Type,Current_Address_Country,Customer_Since_Date
count,100.000000,96,100,100
unique,NaN,2,12,NaN
top,NaN,Personal,UK,NaN
freq,NaN,67,34,NaN
mean,50.500000,NaN,NaN,2023-01-29 09:36:00
min,1.000000,NaN,NaN,2022-01-27 00:00:00
25%,25.750000,NaN,NaN,2022-08-11 06:00:00
50%,50.500000,NaN,NaN,2023-01-17 00:00:00
75%,75.250000,NaN,NaN,2023-09-16 12:00:00
max,100.000000,NaN,NaN,2023-12-26 00:00:00


In [8]:
# There are some null values in the Customer_Type column. 
# Since we can’t fill them, we will add the label “Unknown.”
customer['Customer_Type'].fillna('Unknown', inplace = True)

In [9]:
# Ensure that there are no duplicates in the Customer_Id column
customer_id_counts = customer['Customer_Id'].value_counts()
customer_id_counts[customer_id_counts > 1].count()

0

### Transactions tab

In [10]:
# import DataFrame
transaction = pd.read_excel('Dummy Data.xlsx', sheet_name="Transactions", header=1)

In [11]:
# Repeating the same steps as for customer DataFrame

columns_tr = ['Transaction Id', 
         'Customer_Id ',
         'Amount_GBP',
         'Currency_Route ',
         'Transaction Date'
        ]
transaction = transaction[columns_tr]
transaction = transaction.rename(columns={'Customer_Id ': 'Customer_Id',
                                         'Currency_Route ': 'Currency_Route',
                                         'Transaction Date': 'Transaction_Date',
                                         'Transaction Id': 'Transaction_Id'})

In [12]:
transaction.describe(include='all',datetime_is_numeric=True)

,Transaction_Id,Customer_Id,Amount_GBP,Currency_Route,Transaction_Date
count,90.000000,90.000000,90.000000,90,90
unique,NaN,NaN,NaN,16,NaN
top,NaN,NaN,NaN,GBP --> AUD,NaN
freq,NaN,NaN,NaN,7,NaN
mean,73.477778,53.888889,491.377778,NaN,2022-11-08 17:52:00
min,1.000000,1.000000,-584.000000,NaN,2022-01-08 00:00:00
25%,32.000000,29.000000,176.500000,NaN,2022-05-17 06:00:00
50%,81.000000,55.500000,534.000000,NaN,2022-09-26 00:00:00
75%,108.750000,77.000000,793.500000,NaN,2023-04-26 12:00:00
max,147.000000,100.000000,1000.000000,NaN,2023-12-03 00:00:00


In [13]:
transaction[transaction['Amount_GBP']<=0]

,Transaction_Id,Customer_Id,Amount_GBP,Currency_Route,Transaction_Date
16,109.0,19.0,-250.0,USD --> EUR,2023-09-06
76,107.0,88.0,-584.0,GBP --> USD,2022-08-11


As we can see, there are two negative values in the Amount_GBP column. At the end of this section, these records should be excluded from the report since the nature of the issue is unknown.
(Excluding those two transactions now would be a mistake, as it will affect the issue that will be highlighted next)

In [14]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction_Id    90 non-null     float64       
 1   Customer_Id       90 non-null     float64       
 2   Amount_GBP        90 non-null     float64       
 3   Currency_Route    90 non-null     object        
 4   Transaction_Date  90 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 3.6+ KB


In [15]:
# Changing the type to a more appropriate one, 
# based on the nature of the values in the column.
transaction['Transaction_Id'] = transaction['Transaction_Id'].astype(int)
transaction['Customer_Id'] = transaction['Customer_Id'].astype(int)

In [16]:
transaction_id_counts = transaction['Transaction_Id'].value_counts()
print('Number of transactions with a non-unique Transaction Id:', transaction_id_counts[transaction_id_counts > 1].count())

Number of transactions with a non-unique Transaction Id: 21


In [17]:
transaction[transaction['Transaction_Id']== 11]

,Transaction_Id,Customer_Id,Amount_GBP,Currency_Route,Transaction_Date
2,11,2,253.0,AUD --> AUD,2022-06-30
25,11,31,283.0,GBP --> USD,2022-03-21
43,11,55,36.0,AUD --> GBP,2022-03-24


As we see, there are 21 transactions with a non-unique Transaction_Id, which goes against the given definition stating that Transaction_Id is a unique identifier for a given transaction. Also, as we can see in the presented example, three transactions with Transaction_ID = 11 were made by different customers, with different currency routes, and on different dates, so they can’t be considered duplicates.


The reasons behind this must be further investigated, but for now, they will be removed from the dataset to ensure data integrity.

In [18]:
# Excluding non-unique Transaction Ids.
transactions = transaction[~transaction['Transaction_Id'].duplicated(keep=False)]

# Excluding transaction with a negative amount
transactions = transactions[transactions['Amount_GBP']> 0]


In [19]:
transactions[transactions['Customer_Id']==37]

,Transaction_Id,Customer_Id,Amount_GBP,Currency_Route,Transaction_Date
28,28,37,985.0,EUR --> USD,2023-08-20


In [20]:
print(f'Out of the initial {transaction.Transaction_Id.count()} records in the Transactions DataFrame, only {transactions.Transaction_Id.count()} remain.')

Out of the initial 90 records in the Transactions DataFrame, only 45 remain.


### Merge two dataframes

In [21]:
# As we can see from our previous research, in both DataFrames, 
# Customer_ID values range between 1 and 100. 
# However, to ensure that all transactions are preserved, 
# we will use a left join.
tr_cus = pd.merge(transactions, customer, on='Customer_Id', how='left')

In [22]:
# It is now important to highlight cases where 
# a user’s transaction date was registered before the date they signed up.
tr_cus[tr_cus['Customer_Since_Date'] > tr_cus['Transaction_Date']] #[['Transaction_Date', 'Customer_Since_Date']].head()

,Transaction_Id,Customer_Id,Amount_GBP,Currency_Route,Transaction_Date,Customer_Type,Current_Address_Country,Customer_Since_Date
0,73,1,809.0,GBP --> AUD,2022-08-23,Personal,GBR,2023-02-23
2,85,3,460.0,USD --> USD,2022-12-30,Business,ESP,2023-03-06
5,130,14,547.0,GBP --> EUR,2022-03-08,Personal,UK,2022-12-14
9,19,22,670.0,USD --> GBP,2022-08-24,Personal,UK,2022-12-29
11,144,27,823.0,AUD --> USD,2023-02-05,Business,GBR,2023-10-27
16,12,53,135.0,EUR --> EUR,2023-06-19,Unknown,UK,2023-10-28
17,80,53,538.0,USD --> USD,2022-05-08,Unknown,UK,2023-10-28
23,96,67,853.0,USD --> EUR,2022-02-27,Unknown,FRA,2023-11-24
24,30,72,282.0,EUR --> EUR,2022-05-15,Personal,ESP,2022-11-23
27,1,75,985.0,EUR --> GBP,2022-04-06,Personal,FRA,2022-10-31


In [23]:
print('In the presented data in', tr_cus[tr_cus['Customer_Since_Date'] > tr_cus['Transaction_Date']].Transaction_Id.count(), 'case user’s transaction date was registered before the date they signed up.')

In the presented data in 18 case user’s transaction date was registered before the date they signed up.


In [24]:
# Since this was considered a mistake, 
#these transactions will also be excluded from the analysis.
tr_cu = tr_cus[tr_cus['Customer_Since_Date'] <= tr_cus['Transaction_Date']]

In [25]:
display(tr_cu.head())
print('Number of transactions:', tr_cu.Transaction_Id.count())

,Transaction_Id,Customer_Id,Amount_GBP,Currency_Route,Transaction_Date,Customer_Type,Current_Address_Country,Customer_Since_Date
1,7,2,783.0,AUD --> EUR,2023-12-03,Business,ESP,2023-10-18
3,56,9,573.0,AUD --> EUR,2022-07-22,Personal,DEU,2022-02-12
4,15,12,450.0,GBP --> EUR,2023-03-10,Business,UK,2022-02-06
6,66,16,993.0,EUR --> GBP,2023-02-27,Business,AUS,2022-11-08
7,86,19,71.0,EUR --> AUD,2022-08-30,Unknown,FRA,2022-08-25


Number of transactions: 27


### Conclusion

In this section, two DataFrames were imported: customers and transactions. Several issues were identified, including four customers with missing type information, 21 non-unique but non-duplicated Transaction_Id, and two transactions with negative amounts.

After that the dataframes were merged using the Customer_Id column. Further analysis revealed that in 18 instances, the recorded transaction date preceded the customer’s joining date.

Before proceeding with the data from these records, additional steps are required. It is critical to communicate with the data providers to investigate the root causes of these anomalies and explore potential solutions. Gaining insights from a data steward or the person responsible for this data could clarify these issues and enable the use of the dataset. However, until these anomalies are resolved, it is recommended to exclude the affected data points from further analysis.

## R1
### Total cross-currency volume in GBP under UK Entity in the given date range.
In this part it is needed to present total cross-currency volume in GBP under UK Entity between 01/04/2022 and 01/08/2023. To give answer on this question it will be used dataframe prepared in the previous step. 

In [26]:
tr_cu[(tr_cu['Transaction_Date']>= START_DATE)
       &(tr_cu['Transaction_Date']<= END_DATE)
       &(tr_cu['Current_Address_Country']=='UK')
       &(tr_cu['Currency_Route'].str.count('GBP')==1)
     ].Amount_GBP.sum()

1594.0

## R2
### Total cross-currency volume in GBP under US Entity in the given date range.

In this section, it is necessary to present the total cross-currency volume in GBP under the US Entity between 01/04/2022 and 01/08/2023. To answer this question, we will use the DataFrame prepared in the previous step. Additionally, it is important to pay attention to the given definition of the US Entity. It was mentioned that the US Entity should be considered as transactions from customers whose address country at the time of any transaction was the USA or who made the transaction from the US (IP Address or similar). However, the provided dataset lacks additional information about IP Addresses or similar data. This point should be investigated further before submitting the report.

In [27]:
tr_cu[(tr_cu['Transaction_Date']>= START_DATE)
       &(tr_cu['Transaction_Date']<= END_DATE)
       &(tr_cu['Current_Address_Country']=='USA')
       &(tr_cu['Currency_Route'].str.count('GBP')==1)
     ].Amount_GBP.sum()

0.0

As we can see, the amount is 0, as there were no confirmed transactions under the US Entity.

### Total same-currency volume in GBP under US Entity in the given date range.

According definition of the same-currency transactions in GBP means that the start and end currency is GBP, so in the Currency_Route column GBP should appear twice. 

In [28]:
tr_cu[(tr_cu['Transaction_Date']>= START_DATE)
       &(tr_cu['Transaction_Date']<= END_DATE)
       &(tr_cu['Current_Address_Country']=='USA')
       &(tr_cu['Currency_Route'].str.count('GBP')==2)
     ].Amount_GBP.sum()

0.0

## Alternative Numbers.

As it was shown previously, most of records were excluded from the consideration bacause of the mentioned issues. To get alternative version of the reporting numbers will be used initial transaction dataset.

In [29]:
tr_cus_alt = pd.merge(transaction, customer, on='Customer_Id', how='left')

In [30]:
# Alternative Version of R1 submission
tr_cus_alt[(tr_cus_alt['Transaction_Date']>= START_DATE)
       &(tr_cus_alt['Transaction_Date']<= END_DATE)
       &(tr_cus_alt['Current_Address_Country']=='UK')
       &(tr_cus_alt['Currency_Route'].str.count('GBP')==1)
     ].Amount_GBP.sum()

3520.0

In [31]:
# Alternative Version of R2, part 1 submission
tr_cus_alt[(tr_cus_alt['Transaction_Date']>= START_DATE)
       &(tr_cus_alt['Transaction_Date']<= END_DATE)
       &(tr_cus_alt['Current_Address_Country']=='USA')
       &(tr_cus_alt['Currency_Route'].str.count('GBP')==1)
     ].Amount_GBP.sum()

365.0

In [32]:
# Alternative Version of R2, part 2 submission
tr_cus_alt[(tr_cus_alt['Transaction_Date']>= START_DATE)
       &(tr_cus_alt['Transaction_Date']<= END_DATE)
       &(tr_cus_alt['Current_Address_Country']=='USA')
       &(tr_cus_alt['Currency_Route'].str.count('GBP')==2)
     ].Amount_GBP.sum()

0.0

## Conclusion

### Section 1. R1 & R2
In this section 5 questions to be answered:
1. How would you summarise the results of the data?

Based on the provided data and its analysis, it is clear that the quality of the raw data is currently suboptimal. A significant portion of the records in the dataset cannot be validated, which limits their usability for reporting to the regulators. 

2. How would you summarise main caveats to available data?
The analysis of the available data highlights several significant limitations:

    a.	User Region Changes: The dataset may not account for users changing regions, which could lead to inaccuracies in the report. 
    
	b.	Lack of Time Zone Detail: The dataset does not provide detailed time zone information, which can significantly impact data accuracy. For instance, a transaction logged at a specific time may correspond to different dates in various regions due to time zone differences. This lack of clarity can lead to misinterpretations in the report and affect reporting accuracy.
    
    c. Whole Number Rounding: In the current version of the dataset, all amounts are presented as whole numbers. The method of rounding may also affect accuracy. Best practice in such cases would be to use numbers with two decimal places, as this would provide higher precision.
    
    
3. Are there any issues you identified with the data? Why? If so, how do you propose to fix these?
Yes, several issues have been identified with the data:

	1. Data Formatting Issues: The current format of the dataset requires adjustments, as it does not start from the first row or column, and the column headers lack consistency.
	•	Proposed Fix: Implement a standardized template for data exports that begins from the first row and column, ensuring all headers follow a uniform naming convention. Additionally, a data validation step should be included to identify any missing values.
    
	2. Missing Values in the Customer_Type Column.
	•	Proposed Fix: In this work, missing values were replaced with “Unknown” as this does not affect the final output.
    
	3.	Duplicates in the Transaction_id Column: This is the most significant issue. According to the given definition, Transaction_id must be a unique value, but in the provided dataset, there are many cases where transaction_id is not a unique key for a transaction. As shown, there are not a duplicates of the same transactions but completely different transactions without any patterns. This may indicate significant issues in the data extraction process. Potentially, the entire dataset could represent incorrect figures, leading to incorrect figures in all reported numbers.
	•	Proposed Fix: Reach out to the person responsible for data extraction and clarify the reasons behind the highlighted issue.
    
	4.	Transaction Dates Before User Sign-Up: For some of the presented transactions, the date of the transaction occurs before the user’s sign-up date.
	•	Proposed Fix: This should be considered additional evidence of issues in the data extraction process. The solution lies in communicating with the data extraction responsible person.


4. Is there any information you are missing? If so, what is the nature of missing information? What structure would you require this information in? How would you join it to existing data?

According to the given definitions, a US Entity should be considered as all customers who made a transaction from the US. In the provided dataset, there is no information about this aspect. This information should be part of the metadata for any transaction (e.g., IP address and similar details).

5. What would be your approach given the reports need to be submitted now?

The first and most critical step would be to communicate with stakeholders immediately. In situations where it may not be possible to deliver the work to the expected quality due to data issues or time constraints, the key lies in effective and transparent communication.

This communication should focus on outlining the specific issues, such as missing or unvalidated data, and how they affect the final output.

The goal of this communication is to collaboratively develop an actionable plan. This plan could include:

    1. Reviewing Calculated Numbers: Carefully go through the available calculated metrics to determine if they meet the minimum required standards for reporting.
	2. Proposing interim solutions, such as submitting a partial or provisional report based on the best available data.
	3. Acknowledging any temporary gaps in the report while committing to deliver more accurate data once it is available.

By setting clear expectations and agreeing on the next steps, we can avoid misunderstandings and manage risks. Effective communication ensures that even if the report is not perfect, we maintain transparency and a clear path for improving future submissions.